# Step 2: Location_Query
###### Let's find some locations... a lot of them. 
###### But for the love of all that is holy, do not run this again because running several thousand Google API calls can be EXPENSIVE. 
###### Thank goodness for the free trial credit is all I'm saying. Whew.
-----
Now we're going to cross reference some of the data we exported to CSV from step 1.

Using the centerpoint of the zipcodes we're visualizing, we will perform a nested loop of queries against those coordinates increasing our query range on each pass. (1km, 2km, and 3km). Once we have the returns, we use the Google API ID as a unique identifier to eliminate duplicates from the compiled list.

Through testing, it was found that assembling a master list by running multiple queries, using the google API next-page capability and removing duplicates after completion resulted in the most thourough query results.

As the query structure involves making several hundred api calls: (3 resolutions * 81 zipcodes * 2 api calls = 486), it does take time to run, and ultimately is being saved into a CSV format for later reference wihtout the need for unnecessary additional calls to the Google Places API. The query, run on April 22nd, 2019, returned 3820 unique results with type "Doctor".

Annoyingly, Google did not return those 3800 or so zipcodes along with the rest in the 'nearby' search results. This required a followup loop which pulls down location details on each item in the list by the place_id. This is then used to attach a list to the dataframe, allowing the doctors to be sorted by zipcode for later analysis. This data was exported to CSV.

Once doctors were done, the query was rerun with the place type as hospital. And those results exported as well.

In [2]:
import requests
import pandas as pd
import gmaps
import json
import csv
import time
import os
from keys import gapikey
from area import area
import ast
import re

with open('resources/zip_data.csv', newline='\n') as b:
    ziptgt = pd.read_csv(b)
with open('resources/zip_coords.csv', newline='\n', encoding='utf-8-sig') as c:
    zip_coords = pd.read_csv(c)

In [3]:
ziptgt = ziptgt.set_index('zip')
zip_coords = zip_coords.set_index('ZIP')
zip_df = ziptgt.join(zip_coords, how='left')

zip_df.head()

,city,county,median_household_income,population,feature,area,density,nml_dens,nml_inc,color_key,LAT,LNG
zip,,,,,,,,,,,,
20001,"Washington, DC",District Of Columbia,100447.0,41692,"{'type': 'Feature', 'geometry': {'type': 'Poly...",5.835524,7144.516970,0.759751,0.386274,#0062ce,38.910353,-77.017739
20002,"Washington, DC",District Of Columbia,82022.0,52867,"{'type': 'Feature', 'geometry': {'type': 'Poly...",14.228399,3715.597280,0.964587,0.284281,#0048db,38.905026,-76.983607
20003,"Washington, DC",District Of Columbia,118607.0,26330,"{'type': 'Feature', 'geometry': {'type': 'Poly...",6.447784,4083.573672,0.478169,0.486800,#007cc1,38.881904,-76.990935
20004,"Washington, DC",District Of Columbia,144583.0,1610,"{'type': 'Feature', 'geometry': {'type': 'Mult...",0.904905,1779.192084,0.025057,0.630593,#00a1ae,38.894885,-77.028638
20005,"Washington, DC",District Of Columbia,94506.0,12311,"{'type': 'Feature', 'geometry': {'type': 'Poly...",1.122787,10964.678013,0.221204,0.353387,#005ad2,38.904674,-77.031578


In [4]:
# !!! DO NOT RUN THIS AGAIN (THIS IS A VERY EXPENSIVE QUERY USE THE CSVs) !!!
# places_df = pd.DataFrame(columns=['geometry','icon','id', 'name', 'opening_hours', 'photos', 'place_id', 'plus_code', 'rating', 'reference', 'scope', 'types', 'user_ratings_total', 'vicinity'])

# i = 0
# print("Queries have started. Might be a good time to go get some coffee, it's going to take a few minutes")
# radius_list = [1000, 2000, 3000]
# for radius in radius_list:
#     print(f'Starting {radius} meter scan.')
#     for pzip in list(zip_df.index.values):
#         try:
#             npt = 0
#             target_geo = f"{zip_df['LAT'][pzip]},{zip_df['LNG'][pzip]}"
#             target_type = 'hospital'
#             target_radius = radius

#             params1 = {
#                 "location": target_geo,
#                 "radius": target_radius,
#                 "type": target_type,
#                 "key": gapikey,
#             }

#             base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#             response = requests.get(base_url, params=params1)

#             data1 = response.json()

#             npt = data1['next_page_token']

#             params2 = {
#                 "key": gapikey,
#                 "pagetoken": npt
#             }

#             time.sleep(2)

#             response = requests.get(f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?key={gapikey}&pagetoken={npt}')

#             data2 = response.json()

#             places_df = places_df.append(data1['results'], ignore_index=True)
#             places_df = places_df.append(data2['results'], ignore_index=True)
#             i += 1
#             os.system('clear')
#             print(f'places retrieved for {pzip} | {i} of {len(zip_df.index)} @ {radius} meters')
#             print(f'{len(places_df.index)} places retrieved')
#         except KeyError:
#           print(f'key error, next zip')

#     places_df = places_df.drop_duplicates(subset='id').reset_index(drop=True)
    
# print('Query complete. Thanks for waiting.')

Queries have started. Might be a good time to go get some coffee, it's going to take a few minutes
Starting 1000 meter scan.
key error, next zip
key error, next zip
key error, next zip
key error, next zip


/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


places retrieved for 20005 | 1 of 81 @ 1000 meters
21 places retrieved
places retrieved for 20006 | 2 of 81 @ 1000 meters
58 places retrieved
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
places retrieved for 20036 | 3 of 81 @ 1000 meters
98 places retrieved
places retrieved for 20037 | 4 of 81 @ 1000 meters
127 places retrieved
places retrieved for 20317 | 5 of 81 @ 1000 meters
154 places retrieved
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
key error, next zip
places retrieved for 20814 | 6 of 81 @ 1000

In [7]:
places_geo = pd.DataFrame(columns=['lat','lng','name','vicinity','placeid'])

for p in list(places_df.index.values):
    lat = places_df['geometry'][p]['location']['lat']
    lng = places_df['geometry'][p]['location']['lng']
    vicinity = places_df['vicinity'][p]
    name = places_df['name'][p]
    place_id = places_df['place_id'][p]
    geo_tmp = pd.DataFrame([[lat,lng,name,vicinity,place_id]],columns=['lat','lng','name','vicinity','placeid'])
    places_geo = places_geo.append(geo_tmp).reset_index(drop=True)

places_geo.head()

,lat,lng,name,vicinity,placeid
0,38.896822,-77.032905,Magellan Health Services,"1455 Pennsylvania Avenue Northwest # 400, Wash...",ChIJ52L4Api3t4kRg0pAKmwymv4
1,38.903451,-77.040723,Accel E Medical,"1050 Connecticut Avenue Northwest # 10, Washin...",ChIJL91LFrm3t4kRLXvL7y2Fzug
2,38.908097,-77.041413,"Washington Medical Group, PC","1327 18th Street Northwest, Washington",ChIJIUMgc8e3t4kRhbix8nMNpqM
3,38.903330,-77.039315,Center For Medicare Advocacy,"1025 Connecticut Avenue Northwest # 709, Washi...",ChIJUVhSNLm3t4kRS53v-olIkQ8
4,38.905045,-77.042925,Med Star Health,"1145 19th Street Northwest, Washington",ChIJK5vwSLi3t4kR3z6FlYjKVbo


In [5]:
places_geo.to_csv('resources/hospital_geo_multi.csv', encoding='utf-8')

In [9]:
with open('resources/places_geo_multi.csv', newline='\n') as b:
    geo_places = pd.read_csv(b)
    
places_geo.head()

,lat,lng,name,vicinity,placeid
0,38.896822,-77.032905,Magellan Health Services,"1455 Pennsylvania Avenue Northwest # 400, Wash...",ChIJ52L4Api3t4kRg0pAKmwymv4
1,38.903451,-77.040723,Accel E Medical,"1050 Connecticut Avenue Northwest # 10, Washin...",ChIJL91LFrm3t4kRLXvL7y2Fzug
2,38.908097,-77.041413,"Washington Medical Group, PC","1327 18th Street Northwest, Washington",ChIJIUMgc8e3t4kRhbix8nMNpqM
3,38.903330,-77.039315,Center For Medicare Advocacy,"1025 Connecticut Avenue Northwest # 709, Washi...",ChIJUVhSNLm3t4kRS53v-olIkQ8
4,38.905045,-77.042925,Med Star Health,"1145 19th Street Northwest, Washington",ChIJK5vwSLi3t4kR3z6FlYjKVbo


In [11]:
# !!! DO NOT RUN THIS AGAIN EITHER (THIS IS AN EVEN MORE EXPENSIVE QUERY USE THE CSVs) !!!
# place_zip = []
# expt_zip = []
# i = 1
# for placeid in places_geo['placeid']:
#     params3 = {
#         "placeid":placeid,
#         "key": gapikey,
#     }
#     mtchd = False
#     base_url = "https://maps.googleapis.com/maps/api/place/details/json"
#     response = requests.get(base_url, params=params3)
#     place_details = response.json()
#     print(f"processing {i} of {len(places_geo['placeid'])}")
#     i += 1
#     for addcomp in place_details['result']['address_components']:
#         match = re.match(r'^2\d{4}$', addcomp['short_name'])
#         if match:
#             if mtchd == False:
#                 place_zip.append(match.string)
#                 mtchd = True
#                 mtch1 = match.string
#             else:
#                 expt_zip.append(match.string)
#                 print(f"possible double match {match.string} & {mtch1}")

processing 1 of 429
processing 2 of 429
processing 3 of 429
processing 4 of 429
processing 5 of 429
processing 6 of 429
processing 7 of 429
processing 8 of 429
processing 9 of 429
processing 10 of 429
processing 11 of 429
processing 12 of 429
processing 13 of 429
processing 14 of 429
processing 15 of 429
processing 16 of 429
processing 17 of 429
processing 18 of 429
processing 19 of 429
processing 20 of 429
processing 21 of 429
processing 22 of 429
processing 23 of 429
processing 24 of 429
processing 25 of 429
processing 26 of 429
processing 27 of 429
processing 28 of 429
processing 29 of 429
processing 30 of 429
processing 31 of 429
processing 32 of 429
processing 33 of 429
processing 34 of 429
processing 35 of 429
processing 36 of 429
processing 37 of 429
processing 38 of 429
processing 39 of 429
processing 40 of 429
processing 41 of 429
processing 42 of 429
processing 43 of 429
processing 44 of 429
processing 45 of 429
processing 46 of 429
processing 47 of 429
processing 48 of 429
p

processing 379 of 429
processing 380 of 429
processing 381 of 429
processing 382 of 429
processing 383 of 429
processing 384 of 429
processing 385 of 429
processing 386 of 429
processing 387 of 429
processing 388 of 429
processing 389 of 429
processing 390 of 429
processing 391 of 429
processing 392 of 429
processing 393 of 429
processing 394 of 429
processing 395 of 429
processing 396 of 429
processing 397 of 429
processing 398 of 429
processing 399 of 429
processing 400 of 429
processing 401 of 429
processing 402 of 429
processing 403 of 429
processing 404 of 429
processing 405 of 429
processing 406 of 429
processing 407 of 429
processing 408 of 429
processing 409 of 429
processing 410 of 429
processing 411 of 429
processing 412 of 429
processing 413 of 429
processing 414 of 429
processing 415 of 429
processing 416 of 429
processing 417 of 429
processing 418 of 429
processing 419 of 429
processing 420 of 429
processing 421 of 429
processing 422 of 429
processing 423 of 429
processing

In [22]:
places_geo2 = places_geo
places_geo2['zip'] = place_zip

,lat,lng,name,vicinity,placeid,zip
0,38.896822,-77.032905,Magellan Health Services,"1455 Pennsylvania Avenue Northwest # 400, Wash...",ChIJ52L4Api3t4kRg0pAKmwymv4,20004
1,38.903451,-77.040723,Accel E Medical,"1050 Connecticut Avenue Northwest # 10, Washin...",ChIJL91LFrm3t4kRLXvL7y2Fzug,20036
2,38.908097,-77.041413,"Washington Medical Group, PC","1327 18th Street Northwest, Washington",ChIJIUMgc8e3t4kRhbix8nMNpqM,20036
3,38.903330,-77.039315,Center For Medicare Advocacy,"1025 Connecticut Avenue Northwest # 709, Washi...",ChIJUVhSNLm3t4kRS53v-olIkQ8,20036
4,38.905045,-77.042925,Med Star Health,"1145 19th Street Northwest, Washington",ChIJK5vwSLi3t4kR3z6FlYjKVbo,20036
5,38.899649,-77.027214,Boston Medical Group,"1100 H Street Northwest #650, Washington",ChIJf8As6rm3t4kRuYBP-mt0pQg,20005
6,38.903300,-77.032915,Med Tech,"1025 Vermont Avenue Northwest, Washington",ChIJBXtvgpW3t4kRkddd3Qe9EZ8,20005
7,38.902879,-77.032563,Ehealthd C,"1411 K Street Northwest #300, Washington",ChIJAQAAAPG3t4kRj__6xQKlPNM,20001
8,38.905339,-77.039805,IMA World Health,"1730 M Street Northwest Suite 1100, Washington",ChIJ94znsrm3t4kR3Gjb-h_c4R8,20036
9,38.901704,-77.039762,Cardiocare,"900 17th Street Northwest #1250, Washington",ChIJ361Stbm3t4kRvfPGS6PymWk,20006


In [23]:
places_geo2.to_csv('resources/hospital_geo.csv', encoding='utf-8')
places_geo2.head()

,lat,lng,name,vicinity,placeid,zip
0,38.896822,-77.032905,Magellan Health Services,"1455 Pennsylvania Avenue Northwest # 400, Wash...",ChIJ52L4Api3t4kRg0pAKmwymv4,20004
1,38.903451,-77.040723,Accel E Medical,"1050 Connecticut Avenue Northwest # 10, Washin...",ChIJL91LFrm3t4kRLXvL7y2Fzug,20036
2,38.908097,-77.041413,"Washington Medical Group, PC","1327 18th Street Northwest, Washington",ChIJIUMgc8e3t4kRhbix8nMNpqM,20036
3,38.903330,-77.039315,Center For Medicare Advocacy,"1025 Connecticut Avenue Northwest # 709, Washi...",ChIJUVhSNLm3t4kRS53v-olIkQ8,20036
4,38.905045,-77.042925,Med Star Health,"1145 19th Street Northwest, Washington",ChIJK5vwSLi3t4kR3z6FlYjKVbo,20036


In [24]:
gmaps.configure(api_key=gapikey)

places = places_geo[['lat', 'lng']]

places_layer = gmaps.symbol_layer(
    places, fill_color='red', stroke_color='red', scale=1)
fig = gmaps.figure()
fig.add_layer(places_layer)
fig




Figure(layout=FigureLayout(height='420px'))